In [8]:
import numpy as np

from pydrake.all import (MathematicalProgram, Solve, MonomialBasis,
                         Variables, Polynomial, eq, Substitute, Expression)
import pydrake.symbolic as sym


In [2]:
def create_numpy_array(string):
    # Split the string by space
    values = string.split()
    # Convert the values to float
    return np.array([float(value) for value in values])

w11c_ans_str = "4.5868 -0.0237 0.1742 2.0684 0.1005 2.7412 -0.0038 -0.0333 0.1304 0.2714 2.4268 -0.2897 2.1171 0.0132 6.9634 0.0005 0.0150 -0.0203 0.0001 0.0031 0.0000 0.0001 0.0006 0.0934 0.0116 0.0234 -0.0000 0.0000"
w12c_ans_str = "-1.8328 0.0654 -0.1007 -0.3102 -0.0460 -2.5568 0.0001 0.0177 -0.0427 -0.0515 -0.2710 0.0327 -0.2791 0.0207 -1.3630 -0.0000 -0.0015 0.0048 -0.0120 0.0013 0.0000 0.0000 -0.0001 -0.0092 -0.0054 -0.0031 -0.0000 0.0000"
w22c_ans_str = "7.2139 -0.0124 0.0012 0.0618 0.0954 1.1859 0.0000 -0.0034 0.0088 0.0296 0.0303 -0.0002 0.0377 0.0987 0.4190 0.0000 0.0001 -0.0010 0.0016 -0.0007 -0.0000 0.0000 0.0000 0.0013 0.0012 0.0059 -0.0000 0.0000"
l1c_ans_str = "-3.3514 -0.0118 0.2920 -2.0838 0.1256 -1.6818 0.0136 -0.2138 0.1707 0.0306 -2.6709 0.3296 -2.2965 0.1873 -4.8506 -0.0282 0.2366 -0.1405 0.5670 -0.1170 0.6971 -0.0001 0.0009 -0.0998 -0.0076 -0.0427 0.0003 0.0000"
l2c_ans_str = "0.1711 0.6323 -0.1381 0.2945 -0.4221 0.3728 0.0011 0.0007 0.0482 -0.3245 0.2982 -0.0407 0.2632 -0.3511 0.4786 0.0031 -0.0261 0.0159 -0.0331 0.0474 -0.1364 0.0000 0.0001 0.0097 0.0053 0.0016 0.0001 0.0000"

w11c_ans = create_numpy_array(w11c_ans_str)
w12c_ans = create_numpy_array(w12c_ans_str)
w22c_ans = create_numpy_array(w22c_ans_str)
l1c_ans = create_numpy_array(l1c_ans_str)
l2c_ans = create_numpy_array(l2c_ans_str)

In [15]:
prog = MathematicalProgram()
deg = 6
# Convergence rate = 1-beta
beta = 0.1
# Indeterminates
x = prog.NewIndeterminates(2, 'x_{k}')
u = prog.NewIndeterminates(1, 'u_{k}')
w = prog.NewIndeterminates(4, 'w')

# A and B matrices
Ak = np.array([[1.1-0.1*x[1],   -0.1*x[0]],
               [0.1         ,   0.9]])
Bk = np.array([1, 0])[:, np.newaxis]

# Monomial basis
v = [monomial.ToExpression() for monomial in MonomialBasis(x, deg)]
v_next = [vk.Substitute({x[0]: 1.1*x[0] - 0.1*x[0]*x[1] + u[0],
                         x[1]: 0.9*x[1] + 0.9*x[0]}) for vk in v]
# print("v: ", v)
# print("v_next: ", v_next)
dim_v = len(v)
w11c = prog.NewContinuousVariables(dim_v, 'w11c')
w12c = prog.NewContinuousVariables(dim_v, 'w12c')
w22c = prog.NewContinuousVariables(dim_v, 'w22c')
# print("w11c: ", w11c)

W11k = w11c.dot(v)
W12k = w12c.dot(v)
W22k = w22c.dot(v)
Wk = np.array([[W11k, W12k], [W12k, W22k]])
# print("W11k: ", W11k)


W11k_next = w11c.dot(v_next)
W12k_next = w12c.dot(v_next)
W22k_next = w22c.dot(v_next)
Wk_next = np.array([[W11k_next, W12k_next], [W12k_next, W22k_next]])

r = prog.NewContinuousVariables(1, 'r')

l1c = prog.NewContinuousVariables(dim_v, 'l1c')
l2c = prog.NewContinuousVariables(dim_v, 'l2c')

L1k = l1c.dot(v)
L2k = l2c.dot(v)
Lk = np.array([L1k, L2k])[:, np.newaxis]

print("Wk: ", Wk.shape)
print("Ak: ", Ak.shape)
print("Bk: ", Bk.shape)
print("Lk: ", Lk.shape)

prog.AddLinearCost(r[0])
prog.AddLinearConstraint(r[0] >= 0)

v = np.array(v).reshape(-1, 1)
print("v: ", v.shape)
w = np.array(w).reshape(-1, 1)

cross_diag = Ak @ Wk + Bk @ Lk.T
omega = np.block([[Wk_next, cross_diag],
                 [cross_diag.T, (1-beta)*Wk]])
print("omega: ", omega.shape)
# Note: w is an additional indeterminate that enforces that omega is PSD

prog.AddSosConstraint((w.T @ omega @ w - r[0]).flatten()[0])

# Verify that the solution is meets constraints:
# prog.SetInitialGuess(w11c, w11c_ans)
# prog.SetInitialGuess(w12c, w12c_ans)
# prog.SetInitialGuess(w22c, w22c_ans)
# prog.SetInitialGuess(l1c, l1c_ans)
# prog.SetInitialGuess(l2c, l2c_ans)

result = Solve(prog)
print("Solver succeeded: ", result.is_success())

infeasible_constraints = result.GetInfeasibleConstraints(prog)
for c in infeasible_constraints:
    print(f"infeasible constraint: {c}")

# Extract the solution
print("w11c:\n", result.GetSolution(w11c))
print("\nw12c:\n", result.GetSolution(w12c))
print("\nw22c:\n", result.GetSolution(w22c))
print("\nl1c:\n", result.GetSolution(l1c))
print("\nl2c:\n", result.GetSolution(l2c))

Wk:  (2, 2)
Ak:  (2, 2)
Bk:  (2, 1)
Lk:  (2, 1)
v:  (28, 1)
omega:  (4, 4)
Solver succeeded:  True
w11c:
 [ 0.00000000e+00 -6.16351396e-11  3.43239040e-03 -4.73532106e-04
  6.69182576e-03  2.44077896e-06  5.63388995e-06  1.23779754e-07
  7.96993797e-04  1.93077118e-02 -9.03100005e-03  2.52361741e-03
  3.71158942e-05  3.49970876e+00 -1.36269942e+00  1.80390146e+00
 -1.14381531e+00  7.64042518e-01  2.68775848e-02  1.14901302e-01
  4.88472799e-02 -2.36630423e-02  2.74907938e+00 -7.29101573e-01
  1.67860148e+00  6.11812360e-02  1.76360739e-02  3.17393924e+00]

w12c:
 [ 0.00000000e+00 -4.31904481e-08 -3.71391665e-04 -2.85624820e-04
 -5.49488439e-04 -5.99783432e-06  0.00000000e+00 -3.04936141e-07
 -5.84129231e-04 -1.81221158e-03 -8.31987782e-04 -9.88058560e-04
 -7.36417074e-06 -3.99275094e-01  1.26963905e-01 -1.86549877e-01
  1.46782358e-01 -9.32107751e-02 -1.81299970e-02  2.10052452e-03
 -1.95231286e-02  3.20334761e-03 -2.02735199e+00  1.72095075e-01
 -1.31778674e-01 -6.80684562e-02  1.0876